In [1]:
!pip install pyspark
!pip install bnlp_toolkit
!pip install nlu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=5300f13146743ef68ee931befcbf2cb8fb1b6281aac4746ac8b10be5f8259ef7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
  Obtaining dependency information for bnlp_toolkit from https://files.pythonhosted.org/packages/af/99/5d688fc2e5f6494d6c0a4e6bcdc8192266869caeef3b41dbc3dc8358616d/bnlp_toolkit-4.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for ftfy from https://files.pythonhosted.org/packages/91/f8/dfa32d06cfcbdb76bc46e0f5d69c537de33f4cedb1a15cd4746ab45a6a26/ftfy-6.1.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for wcwidth<0

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bnlp import BengaliCorpus as corpus
from bnlp import CleanText
from sklearn.feature_extraction.text import TfidfVectorizer
from bnlp import NLTKTokenizer
import nlu

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_df = pd.read_csv("/kaggle/input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/Train.csv")
test_df = pd.read_csv("/kaggle/input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/Val.csv")

In [4]:
train_df.head()

,Data,Label
0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,1
1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,2
2,ভাই আপনার কথাই যাদু রয়েছে,1
3,উওরটা আমার অনেক ভাল লেগেছে,1
4,আমার নিজের গাড়ী নিয়ে কি সাজেক যেতে পারবো না ?...,0


In [5]:
longest_string = max(train_df["Data"], key=len)
print(len(longest_string))

372


In [6]:
test_df.head()

,Data,Label
0,আর আমার খুবেই আনন্দ লাকছে ভাইটি চাকরি পেয়েছে,1
1,ভাই আমাদের আগের মেয়র আনিচুল হক নাই যে আমাদের ক...,2
2,আমি মার্ক ওয়েন আর সনির বিশাল ভক্ত । একটা সময় ভ...,1
3,৩ মাস না যেতেই একেকজন ফুলে ফেপে আলুর দম,2
4,"বাংলাদেশের পুলিশ হলো নিরীহ মানুষের যম , আর অত্...",2


In [7]:
longest_string = max(test_df["Data"], key=len)
print(len(longest_string))

288


In [8]:
# from sklearn.model_selection import train_test_split
train = train_df['Data'].values.tolist()
test = test_df['Data'].values.tolist()
train_cluster_id = train_df['Label'].tolist()
test_cluster_id = test_df['Label'].tolist()

In [9]:
X_train = train
y_train = train_cluster_id
X_test = test
y_test = test_cluster_id

In [10]:
pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 57.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 45.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 20.6

In [11]:
import ktrain
from ktrain import text
MODEL_NAME = 'csebuetnlp/banglabert'
t = text.Transformer(MODEL_NAME, maxlen=500)

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [12]:
y_train = [str(x) for x in train_cluster_id]
y_test = [str(x) for x in test_cluster_id]

In [13]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 15
	95percentile : 37
	99percentile : 47


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 37
	99percentile : 47


In [14]:
model = t.get_classifier()

learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)

learner.fit_onecycle(1e-4, 5)



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/5
786/786 [==============================] - 872s 1s/step - loss: 0.7933 - accuracy: 0.6538 - val_loss: 0.6680 - val_accuracy: 0.7167
Epoch 2/5
786/786 [==============================] - 841s 1s/step - loss: 0.6276 - accuracy: 0.7392 - val_loss: 0.7293 - val_accuracy: 0.7269
Epoch 3/5
786/786 [==============================] - 840s 1s/step - loss: 0.5132 - accuracy: 0.8002 - val_loss: 0.6554 - val_accuracy: 0.7466
Epoch 4/5
786/786 [==============================] - 842s 1s/step - loss: 0.3009 - accuracy: 0.8931 - val_loss: 0.7641 - val_accuracy: 0.7466
Epoch 5/5
786/786 [==============================] - 841s 1s/step - loss: 0.1277 - accuracy: 0.9589 - val_loss: 0.8363 - val_accuracy: 0.7543


In [15]:
learner.validate(class_names=t.get_classes())

49/49 [==============================] - 8s 108ms/step
              precision    recall  f1-score   support

           0       0.58      0.55      0.57       354
           1       0.78      0.82      0.80       623
           2       0.83      0.81      0.82       590

    accuracy                           0.75      1567
   macro avg       0.73      0.73      0.73      1567
weighted avg       0.75      0.75      0.75      1567



array([[196,  92,  66],
       [ 82, 509,  32],
       [ 60,  53, 477]])

In [16]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('combined_model')

In [17]:
learner.model.summary()

Model: "tf_electra_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 electra (TFElectraMainLaye  multiple                  110026752 
 r)                                                              
                                                                 
 classifier (TFElectraClass  multiple                  592899    
 ificationHead)                                                  
                                                                 
Total params: 110619651 (421.98 MB)
Trainable params: 110619651 (421.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
